In [220]:
# import dependencies
import pandas as pd
import numpy as np
import re

In [221]:
# load in csv files
world_cups_csv = "Data/WorldCups.csv"
wc_matches_csv = "Data/WorldCupMatches.csv"
wc_players_csv = "Data/WorldCupPlayers.csv"

In [222]:
# read the csv files to data frames
world_cups_df = pd.read_csv(world_cups_csv)
wc_matches_df = pd.read_csv(wc_matches_csv)
wc_players_df = pd.read_csv(wc_players_csv)

### World Cups Data

In [223]:
world_cups_df['Attendance'] = world_cups_df['Attendance'].str.replace('.','').astype(int)

In [224]:
world_cups_df

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1045246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768607
5,1958,Sweden,Brazil,Sweden,France,Germany FR,126,16,35,819810
6,1962,Chile,Brazil,Czechoslovakia,Chile,Yugoslavia,89,16,32,893172
7,1966,England,England,Germany FR,Portugal,Soviet Union,89,16,32,1563135
8,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1603975
9,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1865753


### World Cup Matches

In [225]:
wc_matches_df = wc_matches_df.dropna().reset_index(drop=True)

In [226]:
wc_matches_df = wc_matches_df.drop_duplicates('MatchID').reset_index(drop=True)

In [227]:
wc_matches_df = wc_matches_df.apply(pd.to_numeric, errors='ignore', downcast='integer')

In [228]:
wc_matches_df['Datetime'] = pd.to_datetime(wc_matches_df['Datetime'])

In [229]:
wc_matches_df.insert(2,'Date', '')
wc_matches_df.insert(3,'Time', '')
wc_matches_df['Date'] = [d.date() for d in wc_matches_df['Datetime']]
wc_matches_df['Time'] = [d.time() for d in wc_matches_df['Datetime']]
del wc_matches_df['Datetime']

In [230]:
wc_matches_df.insert(6,'Home Team',wc_matches_df['Home Team Initials'])
wc_matches_df.insert(10,'Away Team',wc_matches_df['Away Team Initials'])
del wc_matches_df['Home Team Name']
del wc_matches_df['Away Team Name']
del wc_matches_df['Away Team Initials']
del wc_matches_df['Home Team Initials']

In [231]:
del wc_matches_df['Half-time Home Goals']
del wc_matches_df['Half-time Away Goals']
del wc_matches_df['RoundID']

In [232]:
wc_matches_df.rename(columns={'MatchID':'MatchID_old'}, inplace=True)
wc_matches_df.insert(0, 'MatchID', wc_matches_df['MatchID_old'])
del wc_matches_df['MatchID_old']

In [233]:
wc_matches_df.head()

,MatchID,Year,Date,Time,Stage,Stadium,City,Home Team,Home Team Goals,Away Team Goals,Away Team,Win conditions,Attendance,Referee,Assistant 1,Assistant 2
0,1096,1930,1930-07-13,15:00:00,Group 1,Pocitos,Montevideo,FRA,4,1,MEX,,4444,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA)
1,1090,1930,1930-07-13,15:00:00,Group 4,Parque Central,Montevideo,USA,3,0,BEL,,18346,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI)
2,1093,1930,1930-07-14,12:45:00,Group 2,Parque Central,Montevideo,YUG,2,1,BRA,,24059,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA)
3,1098,1930,1930-07-14,14:50:00,Group 3,Pocitos,Montevideo,ROU,3,1,PER,,2549,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU)
4,1085,1930,1930-07-15,16:00:00,Group 1,Parque Central,Montevideo,ARG,1,0,FRA,,23409,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU)


### World Cup Players

In [234]:
del wc_players_df['Coach Name']
del wc_players_df['Line-up']
del wc_players_df['Shirt Number']
del wc_players_df['Position']

In [235]:
wc_players_df['Event'] = wc_players_df['Event'].fillna('')

In [236]:
wc_players_df['Goals'] = 0
wc_players_df['Own Goals'] = 0
wc_players_df['Yellow Card'] = 0
wc_players_df['Red Card'] = 0
wc_players_df['Second Yellow Card'] = 0
wc_players_df['Penalty'] = 0
wc_players_df['Missed Penalty'] = 0

In [237]:
for index, row in wc_players_df.iterrows():
    events = row['Event'].split(' ')
    for event in events:
        if len(event) != 0:
            if event[0] == 'G':
                wc_players_df.loc[index,'Goals'] += 1
            elif event[0:2] == 'OG':
                wc_players_df.loc[index,'Own Goals'] += 1
            elif event[0] == 'Y':
                wc_players_df.loc[index,'Yellow Card'] += 1
            elif event[0] == 'R':
                wc_players_df.loc[index,'Red Card'] += 1
            elif event[0:2] == 'SY':
                wc_players_df.loc[index,'Second Yellow Card'] += 1
            elif event[0] == 'P':
                wc_players_df.loc[index,'Penalty'] += 1
            elif event[0:2] == 'MP':
                wc_players_df.loc[index,'Missed Penalty'] += 1

In [238]:
del wc_players_df['Event']
del wc_players_df['RoundID']

In [240]:
wc_players_df.head(7)

,MatchID,Team Initials,Player Name,Goals,Own Goals,Yellow Card,Red Card,Second Yellow Card,Penalty,Missed Penalty
0,1096,FRA,Alex THEPOT,0,0,0,0,0,0,0
1,1096,MEX,Oscar BONFIGLIO,0,0,0,0,0,0,0
2,1096,FRA,Marcel LANGILLER,1,0,0,0,0,0,0
3,1096,MEX,Juan CARRENO,1,0,0,0,0,0,0
4,1096,FRA,Ernest LIBERATI,0,0,0,0,0,0,0
5,1096,MEX,Rafael GARZA,0,0,0,0,0,0,0
6,1096,FRA,Andre MASCHINOT,2,0,0,0,0,0,0


In [13]:
unique_players_df = wc_players_df.drop_duplicates(['Player Name']).reset_index(drop=True)

In [43]:
unique_players_df

,RoundID,MatchID,Team Initials,Player Name,Event
0,201,1096,FRA,Alex THEPOT,
1,201,1096,MEX,Oscar BONFIGLIO,
2,201,1096,FRA,Marcel LANGILLER,G40'
3,201,1096,MEX,Juan CARRENO,G70'
4,201,1096,FRA,Ernest LIBERATI,
5,201,1096,MEX,Rafael GARZA,
6,201,1096,FRA,Andre MASCHINOT,G43' G87'
7,201,1096,MEX,Hilario LOPEZ,
8,201,1096,FRA,Etienne MATTLER,
9,201,1096,MEX,Dionisio MEJIA,
